In [1]:
# importación de librerías
import numpy as np
import cv2
from matplotlib import pyplot as plt # necesario para mostrar imágenes en jupyter

# Ejercicio 1 - Estimación de la matriz cámara a partir de puntos 3D y sus proyecciones 2D.

## Apartado a - Generar una matriz de cámara finita con valores aleatorios en [0,1]

Para que una matriz defina una cámara debe cumplir las siguientes condiciones:

* Ser una matriz $3 \times 4$.

$$ P_{3\times 4} = [M_{3\times 3} | p_{3 \times 1}]$$

* Que el determinante de la submatriz $M_{3 \times 3}$ sea distinto de cero.

$$ det(M) \neq 0$$

En la función implementada, genero una matriz $3 \times 4$ aleatoria inicial, si no cumple la condición de que su determinante de $M$ sea distinto de cero, vuelvo a generar otra matriz aleatoria, hasta conseguir una que cumpla la condición. La función `random.rand` de _Numpy_ nos asegura números aleatorios en el rango $[0,1]$.

In [3]:
def genera_camara_finita():
    P = np.random.rand(3,4)
    while np.linalg.det(P[:3,:3]) == 0:
        P = np.random.rand(3,4)
    return P

P = genera_camara_finita()
P

array([[ 0.75099207,  0.60183256,  0.83787896,  0.04672487],
       [ 0.55364068,  0.49951187,  0.79015998,  0.68726282],
       [ 0.36185467,  0.26598495,  0.37721443,  0.39673858]])

## Apartado b - Generar puntos 3D en dos planos distintos ortogonales.

Para que los puntos generados cumplan la condición de pertenecer a dos planos distintos ortogonales deben tener la siguiente forma:

$$\{(0,x_1,x_2)\;y\;(x_2,x_1,0), \qquad\ donde\;x_1 = [0.1,0.1,1]\;y\;x_2=[0.1,0.1,1]\}$$

es decir, tanto $x_1$ como $x_2$ son todos los números desde el $0,1$ hasta el $1$ en intervalos de $0,1$: $0.1, 0.2, 0.3, \cdots 1$.

In [4]:
# función para generar puntos del mundo 3D con coordenadas {(0, x1, x2) y (x2, x1, 0)}. Es decir, una rejilla de puntos
# en dos planos distintos ortogonales. x1=0.1:0.1:1 y x2=0.1:0.1:1 significa que tenemos que generar valores de x1 y x2
# desde 0.1 a 1 y que aumenten de 0.1 en 0.1.
def genera_puntos_planos_ortogonales_distintos():
    # posibles valores para x1 y x2
    x1 = x2 = np.arange(start=0.1,stop=1,step=0.1,dtype=np.float32)
    # generamos un conjunto de puntos con todas las combinaciones de (x1,x2)
    conjunto = np.concatenate(np.array(np.meshgrid(x1,x2)).T)
    # y le añadimos una columna de ceros al principio
    zeros_vector = np.zeros(conjunto.shape[0])
    conjunto1 = np.hstack((zeros_vector[..., None], conjunto))
    # y otra al final
    conjunto2 = np.hstack((conjunto, zeros_vector[...,None]))

    return np.concatenate((conjunto1, conjunto2))

p = genera_puntos_planos_ortogonales_distintos()
p

array([[ 0.        ,  0.1       ,  0.1       ],
       [ 0.        ,  0.1       ,  0.2       ],
       [ 0.        ,  0.1       ,  0.30000001],
       [ 0.        ,  0.1       ,  0.40000001],
       [ 0.        ,  0.1       ,  0.5       ],
       [ 0.        ,  0.1       ,  0.60000002],
       [ 0.        ,  0.1       ,  0.70000005],
       [ 0.        ,  0.1       ,  0.80000001],
       [ 0.        ,  0.1       ,  0.90000004],
       [ 0.        ,  0.2       ,  0.1       ],
       [ 0.        ,  0.2       ,  0.2       ],
       [ 0.        ,  0.2       ,  0.30000001],
       [ 0.        ,  0.2       ,  0.40000001],
       [ 0.        ,  0.2       ,  0.5       ],
       [ 0.        ,  0.2       ,  0.60000002],
       [ 0.        ,  0.2       ,  0.70000005],
       [ 0.        ,  0.2       ,  0.80000001],
       [ 0.        ,  0.2       ,  0.90000004],
       [ 0.        ,  0.30000001,  0.1       ],
       [ 0.        ,  0.30000001,  0.2       ],
       [ 0.        ,  0.30000001,  0.300

## Apartado c - Proyectar los puntos 3D generados con la cámara P simulada

Para calcular la proyección de un 3D con una cámara $P$ debemos multiplicar la cámara por el punto:

$$x_{3 \times 1} = P_{3 \times 4} \cdot X_{4 \times 1}$$

Debemos convertir el punto 3D $X$ a coordenadas homogéneas (añadiéndole un $1$ como última coordenada) para poder multiplicarlo por la matriz de la cámara. Una vez hecha la multiplicación, obtenemos un vector $x$ con tres elementos, pero un punto 2D sólo tiene dos coordenadas, por lo que debemos eliminar el último elemento del vector:

$$x_x = \frac{x_x}{x_z} \qquad\ x_y = \frac{x_y}{x_z}$$

In [5]:
# función que dado un punto del mundo calcula sus coordenadas de proyección de la cámara.
# Debemos añadirle al punto x un elemento 1 para poder multiplicarlo por la matriz cámara.
camera_projection = lambda x, P: P.dot(np.hstack((x,[1])))

# Proyectar el conjunto de puntos del mundo con la cámara simulada y obtener las coordenadas píxel de su proyección
def proyecta_puntos_en_plano(camara, puntos):
    # definimos el array en el que guardaremos las coordenadas píxel de los puntos
    conjunto = np.zeros(puntos.shape)
    # iteramos sobre el array de puntos del mundo para proyectar los puntos
    for i in range(puntos.shape[0]):
        conjunto[i] = camera_projection(x=puntos[i], P=camara)

    # calculamos las coordenadas píxel diviendo la coordenada x e y por la coordenada z
    coords_pixel = np.zeros((puntos.shape[0], 2))
    for i in range(puntos.shape[0]):
        z = conjunto[i,2]
        coords_pixel[i,0] = conjunto[i,0]/z
        coords_pixel[i,1] = conjunto[i,1]/z

    return coords_pixel

c = proyecta_puntos_en_plano(camara=P, puntos=p)
c

array([[ 0.41360482,  1.77033928],
       [ 0.55031064,  1.79487163],
       [ 0.66779289,  1.81595424],
       [ 0.76984   ,  1.83426696],
       [ 0.85930634,  1.85032202],
       [ 0.93838336,  1.86451268],
       [ 1.00878143,  1.87714586],
       [ 1.071855  ,  1.88846464],
       [ 1.12869071,  1.89866401],
       [ 0.51445847,  1.77620985],
       [ 0.6370021 ,  1.7990787 ],
       [ 0.7431276 ,  1.81888363],
       [ 0.83592731,  1.83620172],
       [ 0.91776288,  1.85147371],
       [ 0.99046902,  1.86504199],
       [ 1.05549272,  1.87717658],
       [ 1.11399046,  1.88809331],
       [ 1.16689782,  1.89796678],
       [ 0.60487936,  1.78147314],
       [ 0.71533863,  1.80288031],
       [ 0.81166634,  1.82154875],
       [ 0.8964113 ,  1.83797244],
       [ 0.97154415,  1.85253329],
       [ 1.03861253,  1.86553124],
       [ 1.09884868,  1.87720509],
       [ 1.15324598,  1.88774736],
       [ 1.2026142 ,  1.89731499],
       [ 0.68640667,  1.78621875],
       [ 0.78647252,

## Apartado d - Implementar el algoritmo DLT

Dados unos puntos 3D, $X$ y sus proyecciones en 2D, $x$, queremos estimar la matriz de la cámara $P$.

$$ x_i = P \cdot X_i \qquad\ i = 1, \ldots, N $$

Como mínimo, $N \geq 6$, ya que cada punto tiene $x$ tiene 3 ecuaciones distintas:

$$ \left[ 
    \begin{array}{ccc}
    0^T & -X_i^T & y_i\cdot X_i^T \\
    X_i^T & 0^T & -x_i\cdot X_i^T \\
    -y_i \cdot X_i^T & x_i \cdot X_i^T & 0^T
    \end{array} \right] \left( \begin{array}{c}
    P^1 \\
    P^2 \\
    P^3
    \end{array} \right) = 0$$

donde cada $P^{iT}$ es un vector de 4 elementos que contiene la $i$-ésima fila de $P$. Debido a que estas tres ecuaciones son linealmente dependientes, podemos quedarnos únicamente con las dos primeras:

$$ \underbrace{\left[ 
    \begin{array}{ccc}
    0^T & -X_i^T & y_i\cdot X_i^T \\
    X_i^T & 0^T & -x_i\cdot X_i^T
    \end{array} \right]}_{M_i} \left( \begin{array}{c}
    P^1 \\
    P^2 \\
    P^3
    \end{array} \right) = 0$$
    
Para cada punto calcularemos su matriz $M_i$ y haremos una matriz $M$ con $2